# **Proyek Analisis Sentimen - Fachreza Riyanda**

In [ ]:
# Import Library
import tweepy as tw
import pandas as pd
import os

# Menggunakan token dari twitter
consumer_key= '2kAfWmPvy5nPcoTOoRoIqSaIM'
consumer_secret= 'hkA4WAIWqS59s30QfMrj0EKJzHVf2bjBdif1HZM5DE6iUkaSX7'
access_token='1539625531591004163-IuSh4LkDgd9wXmyaPvuosVBKUslKv5'
access_token_secret='96bkZCGovhBLmLSVlu48Nlsj5xSp7rSQtvMwRggcopxLf'
bearer_token = 'YOUR_BEARER_TOKEN'  # gunakan Bearer Token untuk API v2

# Autentikasi menggunakan bearer token
client = tw.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# Menentukan query
search_words = "aplikasi pln mobile -is:retweet"
query_params = {
    'query': search_words,
    'tweet.fields': 'created_at,text',
    'max_results': 100  # nilai maksimum per permintaan adalah 100
}

users_locs = []
try:
    response = client.search_recent_tweets(**query_params)
    for tweet in response.data:
        users_locs.append([tweet.created_at, tweet.text])
except Exception as e:
    print(f"Error occurred: {e}")

tweet_text = pd.DataFrame(data=users_locs,
                          columns=["Tanggal", "Tweet Id", "Teks", "Username"])

tweet_text.to_csv('plnmobile.csv', index=False)


In [43]:
import pandas as pd
df= pd.read_csv('/content/plnmobile.csv')
df.head()

,Tanggal,Tweet Id,Teks,Username
0,2022-06-30 21:29:11+00:00,1542621292285030400,@novitamuhayani 3. Pilih ID Pelanggan yang sud...,pln_123
1,2022-06-30 21:29:08+00:00,1542621281795063808,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123
2,2022-06-30 21:28:57+00:00,1542621235481563136,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",pln_123
3,2022-06-30 21:23:24+00:00,1542619836844748800,@dwifortuna34 2. Kakak juga dapat lapor melalu...,pln_123
4,2022-06-30 20:24:12+00:00,1542604938282340352,@pln_123 Di aplikasi sudah tercover kak tapi t...,YBudyyy


In [47]:
df.shape

(10206, 5)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10206 entries, 0 to 10205
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tanggal       10206 non-null  object
 1   Tweet Id      10206 non-null  int64 
 2   Teks          10206 non-null  object
 3   Username      10206 non-null  object
 4   cleaned_text  10206 non-null  object
dtypes: int64(1), object(4)
memory usage: 398.8+ KB


In [48]:
df.dtypes

Tanggal         object
Tweet Id         int64
Teks            object
Username        object
cleaned_text    object
dtype: object

In [45]:
import re

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Menghapus URL
    text = re.sub(r'@\w+', '', text)  # Menghapus mentions
    text = re.sub(r'#\w+', '', text)  # Menghapus hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus tanda baca
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = text.lower()  # Mengubah teks ke huruf kecil
    return text

# Membersihkan teks dalam dataset
df['cleaned_text'] = df['Teks'].apply(clean_text)

# Melihat hasilnya
df[['Teks', 'cleaned_text']].head()

,Teks,cleaned_text
0,@novitamuhayani 3. Pilih ID Pelanggan yang sud...,pilih id pelanggan yang sudah didaftarkan pa...
1,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",hi kak untuk informasi tsb kk dpt melakukan p...
2,"@novitamuhayani Hi Kak, untuk informasi tsb Kk...",hi kak untuk informasi tsb kk dpt melakukan p...
3,@dwifortuna34 2. Kakak juga dapat lapor melalu...,kakak juga dapat lapor melalui aplikasi pln ...
4,@pln_123 Di aplikasi sudah tercover kak tapi t...,di aplikasi sudah tercover kak tapi ternyata ...


In [50]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Contoh fungsi pelabelan sederhana (dapat diganti dengan anotasi manual jika tersedia)
def label_sentiment(text):
    if "tidak" in text or "kurang" in text or "buruk" in text:
        return "negatif"
    elif "baik" in text or "bagus" in text or "puas" in text:
        return "positif"
    else:
        return "netral"

# Melabeli data
df['label'] = df['cleaned_text'].apply(label_sentiment)

# Tokenisasi menggunakan CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['label']

# Memisahkan data menjadi training set dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Melatih model Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Memprediksi hasil pada testing set
y_pred = model.predict(X_test)

# Mengukur akurasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model: {accuracy * 100:.2f}%')

# Melihat distribusi label
df['label'].value_counts()

Akurasi model: 99.51%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


label
netral     8168
positif    1373
negatif     665
Name: count, dtype: int64

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
# Menggunakan TF-IDF untuk ekstraksi fitur
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_text'])

In [53]:
# Memisahkan data menjadi training set dan testing set
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [54]:
# Melatih model Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [55]:
# Memprediksi hasil pada testing set
y_pred_tfidf = rf_model.predict(X_test_tfidf)

In [56]:
# Mengukur akurasi model
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(f'Akurasi model Random Forest dengan TF-IDF: {accuracy_tfidf * 100:.2f}%')

Akurasi model Random Forest dengan TF-IDF: 97.06%


In [57]:
from sklearn.svm import SVC

# Melatih model SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_tfidf, y_train)

# Memprediksi hasil pada testing set
y_pred_svm = svm_model.predict(X_test_tfidf)

# Mengukur akurasi model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'Akurasi model SVM dengan TF-IDF: {accuracy_svm * 100:.2f}%')

Akurasi model SVM dengan TF-IDF: 99.41%
